In [3]:
import pandas as pd
import requests
# Replace 'Total_score.xlsx' with the actual filename
file_path = 'Total_score.xlsx'


merged_df = pd.read_excel(file_path)

In [4]:
import gzip
import os
import time
global last_edit_time
# 定義壓縮檔案路徑
gzip_file_path = 'datasets.jsonl.gz'

# 定義解壓後的檔案路徑
output_file_path = 'datasets.jsonl'

# 解壓縮.gz檔案並儲存為.jsonl檔案
with gzip.open(gzip_file_path, 'rb') as gzip_file:
    file_stat = os.fstat(gzip_file.fileno())
    last_edit_time = file_stat.st_mtime
    last_edit_time_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(last_edit_time))
    with open(output_file_path, 'wb') as output_file:
        while True:
            chunk = gzip_file.read(1024)
            if not chunk:
                break
            output_file.write(chunk)


In [5]:
import json

def process_data(data):
    title = data['title']
    # name = data['name'] #資料集在depositar網址末碼
    keywords = data.get('keywords', [])
    return {title: [keywords]}

result_data = {}



with open(output_file_path, 'r') as jsonl_file:
    for line in jsonl_file:
        data = json.loads(line)
        result_data.update(process_data(data))
        


In [6]:
result_data

{'溫寮溪道路工程(平面道路工區1、堤防道路、橋樑工區1、平面道路工區2、平面道路工區4)施工階段生態檢核成果': [[]],
 '2023-08-05 Nanshan Gravesite Panoramic Images -- Ricoh Theta Z1': [['Q64481569',
   'Q1129474',
   'Q39614',
   'Q22502',
   'Q140631',
   'Q865']],
 '2023-08-05 Nanshan Gravesite Images -- Sony ZV-E10': [['Q64481569',
   'Q1129474',
   'Q39614',
   'Q22502',
   'Q140631',
   'Q865']],
 '三姓溪排水(河心累距0K+760~0K+800)護岸改善應急工程': [[]],
 '高雄市太爺親水運動公園新建計畫': [[]],
 '0802進度報告': [['Q2013',
   'Q1784941',
   'Q554950',
   'Q676880',
   'Q98433008',
   'Q105099901']],
 '台灣生物多樣性網絡物種名錄': [['Q10753560', 'Q38840', 'Q47041']],
 '台灣生物多樣性網絡資料集清單': [['Q1172284', 'Q131449', 'Q104021', 'Q47041']],
 '石門水庫阿姆坪防淤隧道工程計畫-大嵙崁清淤輸送系統委託規畫設計監造生態檢核-[施工階段]': [[]],
 'LTSER-Feitsui 福山水質分析 (避免資料濫用暫時關閉)': [['Q66309937', 'Q625376']],
 'LTSER-Feitsui 金瓜寮水質分析 (避免資料濫用暫時關閉)': [['Q66309937', 'Q625376']],
 'LTSER-Feitsui 福山水文監測資料': [['Q7596765', 'Q190107']],
 'LTSER-Feitsui 福山林內氣象監測資料': [['Q190107']],
 'LTSER-Feitsui 金瓜寮下游氣象監測資料': [['Q190107']],
 'LTSER-Feitsui 金瓜寮上游氣象

In [7]:
def extend(list):
    j=1
    for i in result_data.keys():
        j=j+1
        result_data[i].append([])
        extend_list =[]
        for  ToSearch in result_data[i][0]:
            if ToSearch not in merged_df["QID1"].values :
                # Do something if 'ToSearch' is not present in the 'QID1' column
                top_five_qid2=[]
            else:
                # Filter the data to get rows with the specified QID1 and sort by "Cosine Similarity" in descending order
                filtered_data = merged_df[merged_df["QID1"] == ToSearch].sort_values(by=["W_TotalScore"], ascending=False)
                top_five_qid2 = filtered_data.loc[filtered_data["W_TotalScore"] > 0.775, "QID2"].head(30).tolist()
                extend_list.extend(top_five_qid2)
        #print(len(result_data)-j,end="\t")     
        result_data[i][1].extend(extend_list)
    return result_data

In [8]:
result_data = extend(result_data)

In [9]:
result_data

{'溫寮溪道路工程(平面道路工區1、堤防道路、橋樑工區1、平面道路工區2、平面道路工區4)施工階段生態檢核成果': [[], []],
 '2023-08-05 Nanshan Gravesite Panoramic Images -- Ricoh Theta Z1': [['Q64481569',
   'Q1129474',
   'Q39614',
   'Q22502',
   'Q140631',
   'Q865'],
  ['Q19853609',
   'Q17025454',
   'Q11053261',
   'Q17370635',
   'Q17018836',
   'Q31059',
   'Q712020',
   'Q28417880',
   'Q97173495',
   'Q140631',
   'Q10946672',
   'Q9578202',
   'Q5972740',
   'Q106639055',
   'Q15926827',
   'Q7530041',
   'Q697894',
   'Q34710954',
   'Q77081357',
   'Q7681876',
   'Q75909014',
   'Q64358845',
   'Q11172498',
   'Q17023655',
   'Q75692670',
   'Q63406571',
   'Q15915511',
   'Q10890692',
   'Q11131826',
   'Q60687661',
   'Q66759038',
   'Q1286517',
   'Q210272',
   'Q41176',
   'Q43483',
   'Q331055',
   'Q381885',
   'Q12271',
   'Q159719',
   'Q4022',
   'Q11292',
   'Q93352',
   'Q62832',
   'Q43338',
   'Q4421',
   'Q3947',
   'Q865',
   'Q97173495',
   'Q697926',
   'Q137816',
   'Q29019698',
   'Q918799',
   'Q40285',
  

In [10]:
#00
from collections import Counter
import math
import pandas as pd

# 創建一個空的列表來存儲相似度結果
similarity_results = []

for item1_key in result_data.keys():
    for item2_key in result_data.keys():
        if item1_key != item2_key:
            # 从 result_data 中获取两个项目的第一子列表
            vector1 = result_data[item1_key][0]
            vector2 = result_data[item2_key][0]

            # 将列表转换为频率字典
            vector1_counter = Counter(vector1)
            vector2_counter = Counter(vector2)

            # 计算点积
            dot_product = sum(vector1_counter[x] * vector2_counter[x] for x in vector1_counter if x in vector2_counter)

            # 计算向量的模
            vector1_length = math.sqrt(sum(x ** 2 for x in vector1_counter.values()))
            vector2_length = math.sqrt(sum(x ** 2 for x in vector2_counter.values()))

            # 避免除以零的情況
            if vector1_length != 0 and vector2_length != 0:
                # 计算余弦相似度
                cosine_similarity = dot_product / (vector1_length * vector2_length)
                cosine_similarity = cosine_similarity * 5  # 乘以5只是一个缩放，可根据需要调整
                #print(f"Cosine Similarity between {item1_key} and {item2_key}:", cosine_similarity)
            else:
                cosine_similarity = 0
                #print(cosine_similarity,end="\t")
                #print(f"Vector lengths for {item1_key} and/or {item2_key} are zero, skipping similarity calculation.")
             # 將相似度結果添加到列表中
            
            if cosine_similarity != 0: 
                similarity_results.append([item1_key, item2_key, cosine_similarity])
# 將列表轉換為DataFrame
df = pd.DataFrame(similarity_results, columns=['item1_key', 'item2_key', 'cosine_similarity'])

# 將DataFrame輸出為Excel文件
output_excel_path = 'similarity00_results.xlsx'  # 輸出文件的路徑和名稱
df.to_excel(output_excel_path, index=False)

print(f"DataFrame已輸出為Excel文件: {output_excel_path}")

DataFrame已輸出為Excel文件: similarity00_results.xlsx


In [11]:
#01
from collections import Counter
import math
import pandas as pd

# 創建一個空的列表來存儲相似度結果
similarity_results = []

for item1_key in result_data.keys():
    for item2_key in result_data.keys():
        if item1_key != item2_key:
            # 从 result_data 中获取两个项目的第一子列表
            vector1 = result_data[item1_key][0]
            vector2 = result_data[item2_key][1]

            # 将列表转换为频率字典
            vector1_counter = Counter(vector1)
            vector2_counter = Counter(vector2)

            # 计算点积
            dot_product = sum(vector1_counter[x] * vector2_counter[x] for x in vector1_counter if x in vector2_counter)

            # 计算向量的模
            vector1_length = math.sqrt(sum(x ** 2 for x in vector1_counter.values()))
            vector2_length = math.sqrt(sum(x ** 2 for x in vector2_counter.values()))

            # 避免除以零的情況
            if vector1_length != 0 and vector2_length != 0:
                # 计算余弦相似度
                cosine_similarity = dot_product / (vector1_length * vector2_length)
                cosine_similarity = cosine_similarity * 2  # 乘以5只是一个缩放，可根据需要调整
                #print(f"Cosine Similarity between {item1_key} and {item2_key}:", cosine_similarity)
            else:
                cosine_similarity = 0
                #print(cosine_similarity,end="\t")
                #print(f"Vector lengths for {item1_key} and/or {item2_key} are zero, skipping similarity calculation.")
             # 將相似度結果添加到列表中
            if cosine_similarity != 0: 
                similarity_results.append([item1_key, item2_key, cosine_similarity])
# 將列表轉換為DataFrame
df = pd.DataFrame(similarity_results, columns=['item1_key', 'item2_key', 'cosine_similarity'])

# 將DataFrame輸出為Excel文件
output_excel_path = 'similarity01_results.xlsx'  # 輸出文件的路徑和名稱
df.to_excel(output_excel_path, index=False)

print(f"DataFrame已輸出為Excel文件: {output_excel_path}")

DataFrame已輸出為Excel文件: similarity01_results.xlsx


In [12]:
#10
from collections import Counter
import math
import pandas as pd

# 創建一個空的列表來存儲相似度結果
similarity_results = []

for item1_key in result_data.keys():
    for item2_key in result_data.keys():
        if item1_key != item2_key:
            # 从 result_data 中获取两个项目的第一子列表
            vector1 = result_data[item1_key][1]
            vector2 = result_data[item2_key][0]

            # 将列表转换为频率字典
            vector1_counter = Counter(vector1)
            vector2_counter = Counter(vector2)

            # 计算点积
            dot_product = sum(vector1_counter[x] * vector2_counter[x] for x in vector1_counter if x in vector2_counter)

            # 计算向量的模
            vector1_length = math.sqrt(sum(x ** 2 for x in vector1_counter.values()))
            vector2_length = math.sqrt(sum(x ** 2 for x in vector2_counter.values()))

            # 避免除以零的情況
            if vector1_length != 0 and vector2_length != 0:
                # 计算余弦相似度
                cosine_similarity = dot_product / (vector1_length * vector2_length)
                cosine_similarity = cosine_similarity * 2  # 乘以2只是一个缩放，可根据需要调整
                #print(f"Cosine Similarity between {item1_key} and {item2_key}:", cosine_similarity)
            else:
                cosine_similarity = 0
                #print(cosine_similarity,end="\t")
                #print(f"Vector lengths for {item1_key} and/or {item2_key} are zero, skipping similarity calculation.")
             # 將相似度結果添加到列表中
            if cosine_similarity != 0: 
                similarity_results.append([item1_key, item2_key, cosine_similarity])
# 將列表轉換為DataFrame
df = pd.DataFrame(similarity_results, columns=['item1_key', 'item2_key', 'cosine_similarity'])

# 將DataFrame輸出為Excel文件
output_excel_path = 'similarity10_results.xlsx'  # 輸出文件的路徑和名稱
df.to_excel(output_excel_path, index=False)

print(f"DataFrame已輸出為Excel文件: {output_excel_path}")

DataFrame已輸出為Excel文件: similarity10_results.xlsx


In [13]:
#01
# 創建一個空的列表來存儲相似度結果
similarity_results = []

for item1_key in result_data.keys():
    for item2_key in result_data.keys():
        if item1_key != item2_key:
            # 从 result_data 中获取两个项目的第一子列表
            vector1 = result_data[item1_key][0]
            vector2 = result_data[item2_key][1]

            # 将列表转换为频率字典
            vector1_counter = Counter(vector1)
            vector2_counter = Counter(vector2)

            # 计算点积
            dot_product = sum(vector1_counter[x] * vector2_counter[x] for x in vector1_counter if x in vector2_counter)

            # 计算向量的模
            vector1_length = math.sqrt(sum(x ** 2 for x in vector1_counter.values()))
            vector2_length = math.sqrt(sum(x ** 2 for x in vector2_counter.values()))

            # 避免除以零的情況
            if vector1_length != 0 and vector2_length != 0:
                # 计算余弦相似度
                cosine_similarity = dot_product / (vector1_length * vector2_length)
                cosine_similarity = cosine_similarity * 2  # 乘以2只是一个缩放，可根据需要调整
                #print(f"Cosine Similarity between {item1_key} and {item2_key}:", cosine_similarity)
            else:
                cosine_similarity = 0
                #print(cosine_similarity,end="\t")
                #print(f"Vector lengths for {item1_key} and/or {item2_key} are zero, skipping similarity calculation.")
             # 將相似度結果添加到列表中
            if cosine_similarity != 0: 
                similarity_results.append([item1_key, item2_key, cosine_similarity])
# 將列表轉換為DataFrame
df = pd.DataFrame(similarity_results, columns=['item1_key', 'item2_key', 'cosine_similarity'])

# 將DataFrame輸出為Excel文件
output_excel_path = 'similarity01_results.xlsx'  # 輸出文件的路徑和名稱
df.to_excel(output_excel_path, index=False)

print(f"DataFrame已輸出為Excel文件: {output_excel_path}")

DataFrame已輸出為Excel文件: similarity01_results.xlsx


In [14]:
#11
# 創建一個空的列表來存儲相似度結果
similarity_results = []

for item1_key in result_data.keys():
    for item2_key in result_data.keys():
        if item1_key != item2_key:
            # 从 result_data 中获取两个项目的第一子列表
            vector1 = result_data[item1_key][1]
            vector2 = result_data[item2_key][1]

            # 将列表转换为频率字典
            vector1_counter = Counter(vector1)
            vector2_counter = Counter(vector2)

            # 计算点积
            dot_product = sum(vector1_counter[x] * vector2_counter[x] for x in vector1_counter if x in vector2_counter)

            # 计算向量的模
            vector1_length = math.sqrt(sum(x ** 2 for x in vector1_counter.values()))
            vector2_length = math.sqrt(sum(x ** 2 for x in vector2_counter.values()))

            # 避免除以零的情況
            if vector1_length != 0 and vector2_length != 0:
                # 计算余弦相似度
                cosine_similarity = dot_product / (vector1_length * vector2_length)
                cosine_similarity = cosine_similarity * 1  # 乘以1只是一个缩放，可根据需要调整
                #print(f"Cosine Similarity between {item1_key} and {item2_key}:", cosine_similarity)
            else:
                cosine_similarity = 0
                #print(cosine_similarity,end="\t")
                #print(f"Vector lengths for {item1_key} and/or {item2_key} are zero, skipping similarity calculation.")
             # 將相似度結果添加到列表中
            if cosine_similarity != 0: 
                similarity_results.append([item1_key, item2_key, cosine_similarity])
# 將列表轉換為DataFrame
df = pd.DataFrame(similarity_results, columns=['item1_key', 'item2_key', 'cosine_similarity'])

# 將DataFrame輸出為Excel文件
output_excel_path = 'similarity11_results.xlsx'  # 輸出文件的路徑和名稱
df.to_excel(output_excel_path, index=False)

print(f"DataFrame已輸出為Excel文件: {output_excel_path}")

DataFrame已輸出為Excel文件: similarity11_results.xlsx


In [15]:
file_path00 = 'similarity00_results.xlsx'
file_path01 = 'similarity01_results.xlsx'
file_path10 = 'similarity10_results.xlsx'
file_path11 = 'similarity11_results.xlsx'


merged_df00 = pd.read_excel(file_path00)
print(len(merged_df00))
merged_df01 = pd.read_excel(file_path01)
print(len(merged_df01))
merged_df10 = pd.read_excel(file_path10)
print(len(merged_df10))
merged_df11 = pd.read_excel(file_path11)
print(len(merged_df11))

14490
42788
42788
115002


In [16]:
# Merge the two DataFrames on matching 'QID1' and 'QID2'
merged_df = merged_df11.merge(merged_df10, on=['item1_key', 'item2_key'], how='left')
# Rename the 'Qid_score' column from Table 2 to 'Qid_score_new' to avoid overwriting the original 'Qid_score' column from Table 1
merged_df.rename(columns={'cosine_similarity_x': 'cosine_similarity_11'}, inplace=True)
# Rename the 'Qid_score' column from Table 2 to 'Qid_score_new' to avoid overwriting the original 'Qid_score' column from Table 1
merged_df.rename(columns={'cosine_similarity_y': 'cosine_similarity_10'}, inplace=True)
# Fill any NaN values in the 'Qid_score_new' column with 0
merged_df['cosine_similarity_10'].fillna(0, inplace=True)

# Merge the two DataFrames on matching 'QID1' and 'QID2'
merged_df = merged_df.merge(merged_df01, on=['item1_key', 'item2_key'], how='left')
# Rename the 'Qid_score' column from Table 2 to 'Qid_score_new' to avoid overwriting the original 'Qid_score' column from Table 1
# Merge the two DataFrames on matching 'QID1' and 'QID2'
merged_df = merged_df.merge(merged_df00, on=['item1_key', 'item2_key'], how='left')

# Rename the 'Qid_score' column from Table 2 to 'Qid_score_new' to avoid overwriting the original 'Qid_score' column from Table 1
merged_df.rename(columns={'cosine_similarity_x': 'cosine_similarity_01'}, inplace=True)
# Rename the 'Qid_score' column from Table 2 to 'Qid_score_new' to avoid overwriting the original 'Qid_score' column from Table 1
merged_df.rename(columns={'cosine_similarity_y': 'cosine_similarity_00'}, inplace=True)
# Fill any NaN values in the 'Qid_score_new' column with 0
merged_df['cosine_similarity_01'].fillna(0, inplace=True)
merged_df['cosine_similarity_00'].fillna(0, inplace=True)

In [17]:
merged_df['TotalScore']=merged_df['cosine_similarity_11']+merged_df['cosine_similarity_10']+merged_df['cosine_similarity_01']+merged_df['cosine_similarity_00']
merged_df

,item1_key,item2_key,cosine_similarity_11,cosine_similarity_10,cosine_similarity_01,cosine_similarity_00,TotalScore
0,2023-08-05 Nanshan Gravesite Panoramic Images ...,2023-08-05 Nanshan Gravesite Images -- Sony ZV...,1.000000,0.329914,0.329914,5.0,6.659829
1,2023-08-05 Nanshan Gravesite Panoramic Images ...,台灣生物多樣性網絡物種名錄,0.101015,0.000000,0.000000,0.0,0.101015
2,2023-08-05 Nanshan Gravesite Panoramic Images ...,台灣生物多樣性網絡資料集清單,0.192847,0.000000,0.000000,0.0,0.192847
3,2023-08-05 Nanshan Gravesite Panoramic Images ...,LTSER-Feitsui 福山水質分析 (避免資料濫用暫時關閉),0.047619,0.000000,0.000000,0.0,0.047619
4,2023-08-05 Nanshan Gravesite Panoramic Images ...,LTSER-Feitsui 金瓜寮水質分析 (避免資料濫用暫時關閉),0.047619,0.000000,0.000000,0.0,0.047619
...,...,...,...,...,...,...,...
114997,台江內海書目整理-2014.03.22,臺南古蹟資料,0.136931,0.000000,0.000000,0.0,0.136931
114998,台江內海書目整理-2014.03.22,魍港與麻豆溪地圖,0.933333,0.258199,0.365148,0.0,1.556681
114999,台江內海書目整理-2014.03.22,從龍華教到佛教--光復前臺南德化堂的歷史,0.866667,0.365148,0.365148,0.0,1.596963
115000,台江內海書目整理-2014.03.22,略論日據時期的臺南開元寺,0.866667,0.365148,0.365148,0.0,1.596963


In [18]:
# 將DataFrame輸出為Excel文件
output_excel_path = 'similarity_results.xlsx'  # 輸出文件的路徑和名稱
merged_df.to_excel(output_excel_path, index=False)

print(f"DataFrame已輸出為Excel文件: {output_excel_path}")

DataFrame已輸出為Excel文件: similarity_results.xlsx


In [19]:
# Input the QID to search for
ToSearch = input("請輸入您想要檢索的Dataset：\n")

if ToSearch not in merged_df["item1_key"].values:
    # Do something if 'ToSearch' is not present in the 'QID1' column
    print(f"{ToSearch} not found in the Depositar.")
else:

    # Filter the data to get rows with the specified QID1 and sort by "Cosine Similarity" in descending order
    filtered_data = merged_df[merged_df["item1_key"] == ToSearch].sort_values(by="TotalScore", ascending=False)

    # Set thershold
    top_five_file = filtered_data.loc[filtered_data["TotalScore"] > 1.5, "item2_key"].head(5).tolist()
    
    
    print("=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+")
    #print(f"Qid {ToSearch} 的Label是：{label}")
    print(f"以下是Depositar資料庫內，與「{ToSearch}」相關的資料集Top5:")
    print("=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+")
    # 輸出成Qid：zh-label
    for index, file in enumerate(top_five_file):
        print(f"{index+1}:{file}")
        print("========")



請輸入您想要檢索的Dataset：
 地下水流模型(M值)


=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+
以下是Depositar資料庫內，與「地下水流模型(M值)」相關的關鍵字Top5:
=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+
1:LC81180442015007LGN00
2:臺南公車路線站牌資料
3:LC81180442015087LGN00
4:LC81180442015071LGN00
5:LC81180442015023LGN00
